In [3]:
import import_ipynb
import numpy as np
import pandas as pd
from copy import deepcopy

import tensorflow as tf
import tensorflow_hub as hub

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from transformers import pipeline
from transformers import BertForQuestionAnswering, AutoTokenizer

[nltk_data] Downloading package punkt to /home/agupta44/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/agupta44/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/agupta44/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/agupta44/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [31]:
!pip install huggingface-hub==0.0.12

Defaulting to user installation because normal site-packages is not writeable
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.13.3 requires huggingface-hub<0.1.0,>=0.0.19, but you have huggingface-hub 0.0.12 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [32]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.8.2 requires huggingface-hub==0.0.12, but you have huggingface-hub 0.0.19 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
modelname = 'deepset/bert-base-cased-squad2'

model = BertForQuestionAnswering.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

In [20]:
def get_data_from_file(filename):
    with open(filename) as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
        
    return " ".join(lines), lines

In [21]:
#data, paragraphs = get_data_from_file("harry_potter.txt")

In [24]:
def question2answer(question, context, paragraphs):
    answers = []
    nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)
    answer = nlp({'question': question,
                                  'context': context
                                 })
    print(answer)
    
    for para in paragraphs:
        if answer['answer'] in para:
            answers.append(para)
            
    return answers

In [33]:
# Import libraries
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

datasets = load_dataset("super_glue", "boolq") # Use boolq as dataset for preliminary test

# Load model checkpoint
model_ckpt = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
boolq_enc = datasets.map(lambda x : tokenizer(x['question'], x['passage'], truncation="only_second"), batched=True)
data1 = pd.DataFrame(data= np.c_[boolq_enc["train"]['question'], boolq_enc["train"]['passage']],
                     columns= ['question'] + ['passage'] + ['label'])

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt)

# Define training parameters
batch_size = 8
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1, # Set num_train_epochs to 1 as test
    weight_decay=0.01,
)

from transformers import default_data_collator
data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=boolq_enc["train"],
    eval_dataset=boolq_enc["validation"],
    # data_collator=data_collator, # I had to comment this because it was not working with the default collator
    tokenizer=tokenizer,
)

trainer.train()

Reusing dataset super_glue (/home/agupta44/.cache/huggingface/datasets/super_glue/boolq/1.0.2/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83)


  0%|          | 0/3 [00:00<?, ?it/s]

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/agupta44/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /home/agupta44/.cache/huggingface/trans

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

NameError: name 'pd' is not defined

In [29]:
text = r"""Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi
(fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian
branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan
(officially known as Dari since 1958), and Tajikistan (officially known as Tajiki
since the Soviet era), and some other regions which historically were Persianate
societies and considered part of Greater Iran. It is written in the Persian alphabet,
a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet."""

questions = [
    "do iran and afghanistan speak the same language"
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    # print(inputs["input_ids"].tolist())
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(inputs)
    raw_pred, _, _ = trainer.predict(boolq_enc["train"][0])

***** Running Prediction *****
  Num examples = 7
  Batch size = 8


{'input_ids': tensor([[  101,  1202,   178,  4047,  1105,   170,  2087, 23136, 20300,  2936,
          1103,  1269,  1846,   102,  3886,   113,   120,   100,   117,   118,
           368, 28278,  1179,   120,   114,   117,  1145,  1227,  1118,  1157,
          1322, 10031,  1306,  8040,  5053,   113,   585, 28475, 19775, 28484,
         28506,   175,  9663,  1733,  1182,   113,   100,   114,   113,  5113,
           114,   114,   117,  1110,  1141,  1104,  1103,  2102,  7239,  3483,
          1439,  1103, 11501,   118,  7239,  3392,  1104,  1103, 11501,   118,
          1735,  1846,  1266,   119,  1135,  1110,  3120,  4606,  1107,  3398,
           117,  6469,   113,  3184,  1227,  1112, 23612,  1182,  1290,  2996,
           114,   117,  1105, 26307,   113,  3184,  1227,  1112, 22515,  3454,
          2293,  1290,  1103,  2461,  3386,   114,   117,  1105,  1199,  1168,
          4001,  1134,  8528,  1127,  3886,  2193,  9306,  1105,  1737,  1226,
          1104,  6752,  3398,   119,  

KeyError: 0

In [32]:

print(raw_pred)
boolq_enc["train"][0]['question'] = "Is Harry Potter a wizard?"
boolq_enc["train"][0]['passage'] = "Harry Potter is a wizard."
print(boolq_enc)

[[-0.7560437   0.35092854]
 [-0.48861313  0.03732096]
 [-0.7802408   0.2620712 ]
 ...
 [-0.39867657 -0.23741202]
 [-0.5477288  -0.02837079]
 [-0.04931861 -0.51799   ]]
DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'passage', 'question', 'token_type_ids'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'passage', 'question', 'token_type_ids'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'passage', 'question', 'token_type_ids'],
        num_rows: 3245
    })
})


In [38]:
type(boolq_enc["train"])
import pandas as pd
import numpy as np
data1 = pd.DataFrame(data= np.c_[boolq_enc["train"]['question'], boolq_enc["train"]['passage'], boolq_enc["train"]['label']],
                     columns= ['question'] + ['passage'] + ['label'])

In [39]:
print(data1)

                                               question  \
0       do iran and afghanistan speak the same language   
1     do good samaritan laws protect those who help ...   
2     is windows movie maker part of windows essentials   
3     is confectionary sugar the same as powdered sugar   
4            is elder scrolls online the same as skyrim   
...                                                 ...   
9422             is a us district court a federal court   
9423  can a tenant get a restraining order against a...   
9424       is the golden state warriors in the playoffs   
9425             downton abbey will there be a season 7   
9426  is margin of error the same as confidence inte...   

                                                passage label  
0     Persian language -- Persian (/ˈpɜːrʒən, -ʃən/)...     1  
1     Good Samaritan law -- Good Samaritan laws offe...     1  
2     Windows Movie Maker -- Windows Movie Maker (fo...     1  
3     Powdered sugar -- Powdered su

In [ ]:
def answer2tf()